In [3]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [1]:
!ls -lh

total 7.3G
drwxrwxr-x 4 core core  4.0K Jul 30 15:36 catboost_info
-rw-rw-r-- 1 core core  1.3M Jul 25 11:20 eda.ipynb
-rw-rw-r-- 1 core core  370K Jul 25 05:24 Is-classifier-6-eda.ipynb
-rw-rw-r-- 1 core core  377K Jul 29 05:21 Is-detect-42-eda.ipynb
-rw-rw-r-- 1 core core  164M Jul 26 05:14 merge_0726_1.csv.zip
-rw-rw-r-- 1 core core  148M Jul 28 07:53 merge_0728_1.csv.zip
-rw-rw-r-- 1 core core   17K Jul 28 07:55 merge_df.ipynb
-rw-rw-r-- 1 core core  266M Jul 22 16:57 sub_detect_0722_1.csv
-rw-rw-r-- 1 core core  252M Jul 24 17:01 sub_detect_0724.csv
-rw-rw-r-- 1 core core  157M Jul 25 17:15 sub_is_083vt_0724.csv
-rw-rw-r-- 1 core core  516M Jul 26 04:27 sub_is_42_0726_1.csv
-rw-rw-r-- 1 core core  505M Jul 26 15:20 sub_is_42_0726_2.csv
-rw-rw-r-- 1 core core  501M Jul 29 15:01 sub_is_42_0729.csv
-rw-rw-r-- 1 core core  777M Jul 24 17:06 sub_is_all_label2_0724.csv
-rw-rw-r-- 1 core core  138M Jul 22 17:06 sub_is_th0.01_0723.csv
-rw-rw-r-- 1 core core   33M Jul 22 17:08 sub_is_th0.1

In [4]:
csv_files = ['sub_tuple_rel_0724_2_top35.csv', 'sub_is_42_0726_2.csv',
             #'sub_101_0-50_100boxes_0709_1.csv', 
             #'sub_101_400-500_100boxes_0711_1.csv'
            ]

dfs = [pd.read_csv(x) for x in csv_files]

In [5]:
dfs[0].head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.7382881 /m/04yx4 0.0061979 0.1177801 0.40560...
1,763c72626bf24534,0.47459640000000003 /m/03bt1vf 0.3140645 0.210...
2,7540c162180973b1,0.31570315 /m/04yx4 0.9497286 0.3292059 0.9990...
3,c4e522dce52cb02f,0.49188625 /m/04yx4 0.0016701 0.1656198 0.2330...
4,cafcd9496c808085,0.18966015 /m/01_5g 0.7972641 0.0216606 0.9990...


In [6]:
dfs[1].head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.3806741 /m/04bcr3 0.0031277 0.7127081 0.4623...
1,763c72626bf24534,0.7894505 /m/04bcr3 0.6918395 0.8270421 0.9988...
2,7540c162180973b1,0.0402823 /m/04bcr3 0.3689175 0.6536201 0.6134...
3,c4e522dce52cb02f,0.7901130 /m/04bcr3 0.8337175 0.6788673 0.9917...
4,cafcd9496c808085,0.1362440 /m/04dr76w 0.0390787 0.0196190 0.961...


In [7]:
x = None
for d in dfs:
    if x is None:
        x = d.PredictionString
    else:
        x = x.str.cat(d.PredictionString, sep=' ')

In [8]:
dfs[0].PredictionString = x
print(len(dfs[0].iloc[0].PredictionString), len(dfs[1].iloc[0].PredictionString))

13731 5602


In [9]:
dfs[0].head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.7382881 /m/04yx4 0.0061979 0.1177801 0.40560...
1,763c72626bf24534,0.47459640000000003 /m/03bt1vf 0.3140645 0.210...
2,7540c162180973b1,0.31570315 /m/04yx4 0.9497286 0.3292059 0.9990...
3,c4e522dce52cb02f,0.49188625 /m/04yx4 0.0016701 0.1656198 0.2330...
4,cafcd9496c808085,0.18966015 /m/01_5g 0.7972641 0.0216606 0.9990...


In [10]:
dfs[0].to_csv('merge_0724_2.csv.zip', index=False, compression='zip')

In [11]:
!kaggle competitions submit -c open-images-2019-visual-relationship -f merge_0724_2.csv.zip -m "submit"

100%|████████████████████████████████████████| 181M/181M [00:15<00:00, 12.6MB/s]
Successfully submitted to Open Images 2019 - Visual Relationship

In [ ]:
!ls

In [100]:
#####
lst = ['Geeks', 'For', 'Geeks', 'is',  
            'portal', 'for', 'Geeks'] 
tmp1 = pd.DataFrame(lst, columns=['t'])
lst2 = ['a', 'b', 'c', 'd',  
            'e', 'f', 'g'] 
tmp2 = pd.DataFrame(lst2, columns=['t'])

lst3 = ['a1', 'b1', 'c', 'd',  
            'e', 'f', 'g'] 
tmp3 = pd.DataFrame(lst3, columns=['t'])
tmp3.head()

,t
0,a1
1,b1
2,c
3,d
4,e


In [102]:
dfs = [tmp1, tmp2, tmp3]
x = None
for d in dfs:
    if x is None:
        x = d.t
    else:
        x = x.str.cat(d.t, sep=' ')

#x = tmp1.t1.str.cat(tmp2.t2, sep=' ').str.cat(tmp3.t3, sep=' ')
print(x)
tmp1.t = x
tmp1.head()

0    Geeks a a1
1      For b b1
2     Geeks c c
3        is d d
4    portal e e
5       for f f
6     Geeks g g
Name: t, dtype: object


,t
0,Geeks a a1
1,For b b1
2,Geeks c c
3,is d d
4,portal e e
